# Naive Bayes

Naive Bayes is a probabilistic classifier that's embarrassingly simple to train, it is literally just counting, yet works surprisingly well in practice, especially for text.

The "naive" part refers to a simplifying assumption about feature independence. This assumption is almost always wrong, but the algorithm works "well enough" most of the times. Understanding *why*, requires us to dig into probability theory.

**What we'll cover:**
- Generative vs discriminative models
- Bayes' theorem (properly derived)
- The conditional independence assumption
- Training by counting
- Laplace smoothing
- Different variants (Gaussian, Multinomial, Bernoulli)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups
import seaborn as sns

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')

## Generative vs Discriminative Models

There are two fundamentally different approaches to classification:

**Discriminative models** (like logistic regression) learn $P(Y|X)$ directly:
- "Given these features, what's the probability of each class?"
- Learn the decision boundary between classes
- Don't model how data is generated

**Generative models** (like Naive Bayes) learn $P(X|Y)$ and $P(Y)$:
- "What do features look like for each class?"
- Model the distribution of data within each class
- Can generate synthetic examples (in principle)

Naive Bayes is generative: it learns what spam emails look like, what ham emails look like, then uses Bayes' theorem to flip things around for prediction.

## Bayes' Theorem: The Foundation

For classification, we want $P(Y|X)$, the probability of class $Y$ given features $X$.

**Bayes' theorem** lets us express this in terms of things we can estimate:

$$P(Y|X) = \frac{P(X|Y) \cdot P(Y)}{P(X)}$$

Let's understand each term:

| Term | Name | Meaning |
|------|------|--------|
| $P(Y|X)$ | **Posterior** | What we want: probability of class given features |
| $P(X|Y)$ | **Likelihood** | How likely these features are for this class |
| $P(Y)$ | **Prior** | How common this class is overall |
| $P(X)$ | **Evidence** | How common these features are (any class) |

## From Bayes' Theorem to Classification

For classification, we compare $P(Y=c|X)$ across all classes $c$ and pick the highest.

**Key insight:** The denominator $P(X)$ is the same for all classes!

$$P(Y=\text{spam}|X) = \frac{P(X|Y=\text{spam}) \cdot P(Y=\text{spam})}{P(X)}$$

$$P(Y=\text{ham}|X) = \frac{P(X|Y=\text{ham}) \cdot P(Y=\text{ham})}{P(X)}$$

Since we're only comparing (not computing exact probabilities), we can ignore $P(X)$:

$$\hat{Y} = \arg\max_c \; P(X|Y=c) \cdot P(Y=c)$$

## The Problem: High-Dimensional Features

If $X = (X_1, X_2, ..., X_d)$ is a vector of $d$ features, we need:

$$P(X_1, X_2, ..., X_d | Y=c)$$

This is the joint distribution of all features for class $c$. 

**Problem:** Estimating joint distributions requires exponentially many parameters!

Example: 1000 binary features → $2^{1000}$ possible combinations: tanti auguri.

We need a simplifying assumption...

## The Naive Assumption: Conditional Independence

**Assumption:** Features are conditionally independent given the class.

$$P(X_1, X_2, ..., X_d | Y) = P(X_1|Y) \cdot P(X_2|Y) \cdot ... \cdot P(X_d|Y) = \prod_{i=1}^{d} P(X_i|Y)$$

**What this means:** Once you know the class, knowing one feature tells you nothing about another.

**Example (spam detection):**
- If an email is spam, the probability it contains "free" doesn't depend on whether it contains "winner"
- This is obviously false! Spam emails with "free" are more likely to also have "winner"
- But the assumption makes the math tractable

### The Complete Model

Combining everything:

$$\hat{Y} = \arg\max_c \; P(Y=c) \prod_{i=1}^{d} P(X_i|Y=c)$$

This is the **Naive Bayes classifier**.

## Training: Just Counting

Both components are estimated by counting:

**Prior probability:**
$$P(Y=c) = \frac{\text{number of examples in class } c}{\text{total examples}}$$

**Likelihood (for categorical features):**
$$P(X_i = v | Y = c) = \frac{\text{count of examples where } X_i=v \text{ and } Y=c}{\text{count of examples where } Y=c}$$

That's it. No optimization, no gradient descent. Just counting.

In [4]:
# Complete worked example: Email spam classification
# Simplified dataset with word presence/absence

print("Training Data: Email Classification")
print("=" * 60)
print()

# Training data: [has_free, has_winner, has_meeting, has_report]
emails = [
    # Spam emails
    {'free': 1, 'winner': 1, 'meeting': 0, 'report': 0, 'label': 'spam'},
    {'free': 1, 'winner': 0, 'meeting': 0, 'report': 0, 'label': 'spam'},
    {'free': 1, 'winner': 1, 'meeting': 0, 'report': 0, 'label': 'spam'},
    {'free': 0, 'winner': 1, 'meeting': 0, 'report': 0, 'label': 'spam'},
    # Ham emails  
    {'free': 0, 'winner': 0, 'meeting': 1, 'report': 1, 'label': 'ham'},
    {'free': 0, 'winner': 0, 'meeting': 1, 'report': 0, 'label': 'ham'},
    {'free': 1, 'winner': 0, 'meeting': 1, 'report': 1, 'label': 'ham'},
    {'free': 0, 'winner': 0, 'meeting': 0, 'report': 1, 'label': 'ham'},
    {'free': 0, 'winner': 0, 'meeting': 1, 'report': 1, 'label': 'ham'},
    {'free': 0, 'winner': 0, 'meeting': 1, 'report': 0, 'label': 'ham'},
]

# Count
n_spam = sum(1 for e in emails if e['label'] == 'spam')
n_ham = sum(1 for e in emails if e['label'] == 'ham')
n_total = len(emails)

print(f"Total emails: {n_total}")
print(f"  Spam: {n_spam} ({n_spam/n_total:.0%})")
print(f"  Ham:  {n_ham} ({n_ham/n_total:.0%})")

# Priors
p_spam = n_spam / n_total
p_ham = n_ham / n_total

print(f"\nPrior probabilities:")
print(f"  P(spam) = {n_spam}/{n_total} = {p_spam:.2f}")
print(f"  P(ham)  = {n_ham}/{n_total} = {p_ham:.2f}")

Training Data: Email Classification

Total emails: 10
  Spam: 4 (40%)
  Ham:  6 (60%)

Prior probabilities:
  P(spam) = 4/10 = 0.40
  P(ham)  = 6/10 = 0.60


In [5]:
# Calculate likelihoods for each word
words = ['free', 'winner', 'meeting', 'report']

print("Likelihoods P(word=1 | class):")
print("=" * 60)

likelihoods = {'spam': {}, 'ham': {}}

for word in words:
    # Count word occurrences in each class
    count_in_spam = sum(1 for e in emails if e['label'] == 'spam' and e[word] == 1)
    count_in_ham = sum(1 for e in emails if e['label'] == 'ham' and e[word] == 1)
    
    p_word_given_spam = count_in_spam / n_spam
    p_word_given_ham = count_in_ham / n_ham
    
    likelihoods['spam'][word] = p_word_given_spam
    likelihoods['ham'][word] = p_word_given_ham
    
    print(f"\n'{word}':")
    print(f"  P('{word}'=1 | spam) = {count_in_spam}/{n_spam} = {p_word_given_spam:.2f}")
    print(f"  P('{word}'=1 | ham)  = {count_in_ham}/{n_ham} = {p_word_given_ham:.2f}")

Likelihoods P(word=1 | class):

'free':
  P('free'=1 | spam) = 3/4 = 0.75
  P('free'=1 | ham)  = 1/6 = 0.17

'winner':
  P('winner'=1 | spam) = 3/4 = 0.75
  P('winner'=1 | ham)  = 0/6 = 0.00

'meeting':
  P('meeting'=1 | spam) = 0/4 = 0.00
  P('meeting'=1 | ham)  = 5/6 = 0.83

'report':
  P('report'=1 | spam) = 0/4 = 0.00
  P('report'=1 | ham)  = 4/6 = 0.67


In [6]:
# Classify a new email
new_email = {'free': 1, 'winner': 0, 'meeting': 1, 'report': 0}

print("Classifying new email:")
print(f"  Words present: {[w for w, v in new_email.items() if v == 1]}")
print(f"  Words absent:  {[w for w, v in new_email.items() if v == 0]}")
print("\n" + "=" * 60)

def compute_score(label, email, priors, likelihoods):
    """Compute P(label) * product of P(features | label)"""
    score = priors[label]
    terms = [f"P({label}) = {priors[label]:.2f}"]
    
    for word, present in email.items():
        if present == 1:
            p = likelihoods[label][word]
            terms.append(f"P('{word}'=1|{label}) = {p:.2f}")
        else:
            p = 1 - likelihoods[label][word]
            terms.append(f"P('{word}'=0|{label}) = {p:.2f}")
        score *= p
    
    return score, terms

priors = {'spam': p_spam, 'ham': p_ham}

# Compute for spam
score_spam, terms_spam = compute_score('spam', new_email, priors, likelihoods)
print("\nP(spam | email) is proportional to:")
print(f"  {' × '.join(terms_spam)}")
print(f"  = {score_spam:.6f}")

# Compute for ham
score_ham, terms_ham = compute_score('ham', new_email, priors, likelihoods)
print("\nP(ham | email) is proportional to:")
print(f"  {' × '.join(terms_ham)}")
print(f"  = {score_ham:.6f}")

# Normalize to get actual probabilities
total = score_spam + score_ham
prob_spam = score_spam / total
prob_ham = score_ham / total

print("\n" + "=" * 60)
print(f"Normalized probabilities:")
print(f"  P(spam | email) = {score_spam:.6f} / {total:.6f} = {prob_spam:.1%}")
print(f"  P(ham | email)  = {score_ham:.6f} / {total:.6f} = {prob_ham:.1%}")
print(f"\nPrediction: {'SPAM' if prob_spam > prob_ham else 'HAM'}")

Classifying new email:
  Words present: ['free', 'meeting']
  Words absent:  ['winner', 'report']


P(spam | email) is proportional to:
  P(spam) = 0.40 × P('free'=1|spam) = 0.75 × P('winner'=0|spam) = 0.25 × P('meeting'=1|spam) = 0.00 × P('report'=0|spam) = 1.00
  = 0.000000

P(ham | email) is proportional to:
  P(ham) = 0.60 × P('free'=1|ham) = 0.17 × P('winner'=0|ham) = 1.00 × P('meeting'=1|ham) = 0.83 × P('report'=0|ham) = 0.33
  = 0.027778

Normalized probabilities:
  P(spam | email) = 0.000000 / 0.027778 = 0.0%
  P(ham | email)  = 0.027778 / 0.027778 = 100.0%

Prediction: HAM


## The Zero-Frequency Problem

What if a word never appears in spam during training, but appears in a test email?

$$P(\text{word}|\text{spam}) = \frac{0}{n_{\text{spam}}} = 0$$

Since we multiply probabilities:

$$P(\text{spam}|\text{email}) \propto P(\text{spam}) \times ... \times 0 \times ... = 0$$

One unseen word zeros out the entire probability! This is catastrophic.

### Solution: Laplace Smoothing

Add a small count $\alpha$ (typically 1) to every feature count:

$$P(X_i = v | Y = c) = \frac{\text{count}(X_i = v, Y = c) + \alpha}{\text{count}(Y = c) + \alpha \cdot |V|}$$

Where $|V|$ is the number of possible values for feature $X_i$.

**Effect:** No probability is ever exactly 0 or 1.

**Note:**  $\alpha \cdot |V|$ at the denominator allows normalization, effectively mantaining the score a probability.

## Log Probabilities: Avoiding Underflow

Another practical issue: multiplying many small probabilities causes numerical underflow.

$$P(Y|X) \propto P(Y) \prod_{i=1}^{d} P(X_i|Y)$$

With 1000 features and $P(X_i|Y) \approx 0.01$, we'd compute $0.01^{1000} \approx 10^{-2000}$.

**Solution:** Work in log space.

$$\log P(Y|X) \propto \log P(Y) + \sum_{i=1}^{d} \log P(X_i|Y)$$

Products become sums, which are numerically stable. We compare log-probabilities instead.

## When Does The Naive Assumption Fail?

The independence assumption is almost always violated. When does it matter?

**Usually okay:**
- Text classification (words have complex dependencies, but NB still works)
- When you just need a ranking (relative probabilities), not calibrated probabilities
- High-dimensional sparse data

**Problematic:**
- Duplicate or highly correlated features (double-counting evidence)
- When you need accurate probability estimates
- Low-dimensional data where other models work better anyway

## Summary

| Concept | Description |
|---------|-------------|
| **Core idea** | Use Bayes' theorem: $P(Y|X) \propto P(X|Y) \cdot P(Y)$ |
| **Naive assumption** | Features independent given class: $P(X|Y) = \prod_i P(X_i|Y)$ |
| **Training** | Count frequencies to estimate $P(Y)$ and $P(X_i|Y)$ |
| **Prediction** | Pick class with highest $P(Y) \prod_i P(X_i|Y)$ |
| **Laplace smoothing** | Add $\alpha$ to counts to avoid zero probabilities |
| **Log probabilities** | Use sums of logs to avoid numerical underflow |

**Variants:**
- Gaussian: continuous features, model as normal distribution
- Multinomial: count features (word counts in text)
- Bernoulli: binary features (word presence/absence)

**When to use:**
- Text classification (spam, sentiment, topic)
- High-dimensional sparse data
- Need a fast baseline
- Limited training data

**Limitations:**
- Independence assumption often wrong
- Probability estimates can be poorly calibrated
- Correlated features cause problems